In [1]:
from utz import *
from nj_crashes.paths import ROOT_DIR
from njsp import crashes
from njsp.paths import PROJECTED_CSV
from njsp.ytc import to_ytc
from njsp.ytd import oldest_commit_rundate_since, projected_roy_deaths
from njsp import Ytd

chdir(ROOT_DIR)

In [2]:
ytd = Ytd()
prv_rundate = ytd.prv_rundate
prv_rundate

Searching for oldest commit with rundate ≥2024-07-09


Found rundate 2024-07-08 10:00:05 < 2024-07-09 at commit 8734370f; returning commit 504b241


'2024-07-09 10:00:03'

In [3]:
prv_ytd_ratio = ytd.cur_ytd_deaths / ytd.prv_ytd_deaths
pct_change = (prv_ytd_ratio - 1) * 100

print(f'As of {ytd.prv_rundate}, NJSP was reporting {ytd.prv_ytd_total} YTD deaths')
print(f'Current YTD Deaths ({ytd.rundate}): {ytd.cur_ytd_deaths}')
print(f'Previous year YTD Deaths (adjusted; {ytd.prv_rundate}): {ytd.prv_ytd_deaths}')
print(f'Projected {ytd.cur_year} total: {ytd.projected_year_total}')
print(f'{pct_change:.1f}% change')

As of 2024-07-09 10:00:03, NJSP was reporting 323 YTD deaths
Current YTD Deaths (2025-07-09 10:00:02-04:00): 280
Previous year YTD Deaths (adjusted; 2024-07-09 10:00:03): 322.18361242030187
Projected 2025 total: 620.9659916407905
-13.1% change


afd494565d8239140726518dd90b5df93c4ba991: FAUQStats cache miss: 2024, Tue Jul 09 10:00:03 EDT 2024


Find the oldest commit with rundate less than 1 year ago, load crashes from that time:

In [4]:
prv_crashes = ytd.prv_ytd_crashes
prv_crashes

,CCODE,CNAME,MCODE,MNAME,STREET,LOCATION,FATALITIES,FATAL_D,FATAL_P,FATAL_T,FATAL_B,HIGHWAY,INJURIES,dt
ACCID,,,,,,,,,,,,,,
12918,12,Middlesex,1218,Plainsboro Twsp,NaN,State Highway 1,1.0,0.0,0.0,1.0,0.0,1,NaN,2024-01-01 01:56:00-05:00
12923,11,Mercer,1111,Trenton,NaN,State Highway 129,1.0,1.0,0.0,0.0,0.0,129,NaN,2024-01-03 12:14:00-05:00
12932,01,Atlantic,0112,Hamilton Twsp,NaN,Atlantic City Expressway MP 20.5,1.0,1.0,0.0,0.0,0.0,446,NaN,2024-01-05 09:14:00-05:00
12933,02,Bergen,0248,Ramsey Boro,East Oak St,East Oak St,1.0,1.0,0.0,0.0,0.0,NaN,NaN,2024-01-05 14:12:00-05:00
12927,20,Union,2009,Linden City,NaN,State Highway 27,1.0,0.0,0.0,1.0,0.0,27,NaN,2024-01-06 08:37:00-05:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13288,21,Warren,2107,Greenwich Twsp,NaN,Interstate 78 MP 5.1,1.0,1.0,0.0,0.0,0.0,78,0.0,2024-07-02 22:04:00-04:00
13287,13,Monmouth,1312,Englishtown Boro,NaN,County 527,1.0,1.0,0.0,0.0,0.0,527,NaN,2024-07-04 01:52:00-04:00
13291,02,Bergen,0246,Paramus Boro,NaN,Garden State Parkway MP 163.4,2.0,2.0,0.0,0.0,0.0,444,1.0,2024-07-05 02:27:00-04:00


Load previous year YTD and final counts, and current year YTD counts:

In [5]:
prv_year = ytd.prv_year
cur_year = ytd.cur_year

prv_ytc = to_ytc(prv_crashes)
prv_ytd = prv_ytc.loc[prv_year]

prv_end = to_ytc(ytd.prv_end_crashes).loc[prv_year]

cur_ytc = to_ytc(ytd.cur_ytd_crashes)
cur_ytd = cur_ytc.loc[cur_year]

cur_ytd

,driver,passenger,pedestrian,cyclist,crashes
county,,,,,
Atlantic,11,2,5,0,17
Bergen,8,1,2,0,11
Burlington,14,4,3,0,20
Camden,11,2,8,0,20
Cape May,5,1,0,0,6
Cumberland,6,2,3,0,10
Essex,6,3,11,1,21
Gloucester,9,4,2,0,12
Hudson,2,1,3,1,6


Compute fraction of current year that has elapsed (year-to-date; "YTD") and still remains (rest-of-year; "ROY"):

In [6]:
cur_ytd_frac = ytd.cur_year_frac
cur_roy_frac = 1 - cur_ytd_frac
print('%.1f%% through the year, %.1f%% remaining' % (ytd.cur_year_frac * 100, ytd.cur_roy_frac * 100))

51.9% through the year, 48.1% remaining


Combine previous year YTD / end with current year YTD:

In [7]:
def melt(df, name):
    return (
        df
        .melt(ignore_index=False, var_name='type')
        .set_index('type', append=True)
        .value
        .rename(name)
    )

z = sxs(
    melt(prv_ytd, 'prv_ytd'),
    melt(prv_end, 'prv_end'),
    melt(cur_ytd, 'cur_ytd'),
).fillna(0).astype(int)
z

,,prv_ytd,prv_end,cur_ytd
county,type,,,
Atlantic,driver,14,31,11
Bergen,driver,11,14,8
Burlington,driver,18,31,14
Camden,driver,11,19,11
Cape May,driver,2,8,5
...,...,...,...,...
Salem,crashes,8,14,9
Somerset,crashes,7,18,7
Sussex,crashes,6,12,2


In [8]:
def project_roy(r):
    return int(round(projected_roy_deaths(r.prv_ytd, r.prv_end, r.cur_ytd, cur_ytd_frac)))

roy = z.apply(project_roy, axis=1).rename('roy')
projected = (
    (z.cur_ytd + roy)
    .rename('projected')
    .reset_index(level=1)
    .pivot(columns='type', values='projected')
)
projected

type,crashes,cyclist,driver,passenger,pedestrian
county,,,,,
Atlantic,38,0,26,4,11
Bergen,25,0,11,2,10
Burlington,39,0,26,5,12
Camden,32,0,19,3,12
Cape May,20,0,16,1,0
Cumberland,23,0,15,6,8
Essex,45,1,16,10,24
Gloucester,35,0,22,13,6
Hudson,25,1,10,1,17


In [9]:
projected.to_csv(PROJECTED_CSV)